# Predicates

In ACES, predicates specify how particular concepts relevant to your task of interest is expressed in your dataset of interest. These dataset-specific items form a large foundation of the cohort extraction algorithm as the more complex dataset-agnostic windowing logic of your task is defined based on your predicates, ultimately facilitating ease-of-sharing for your task configurations.

## Predicate Columns

A predicate column is simply a column in a dataframe containing numerical counts (often just `0`'s and `1`'s), representing the number of times a given predicate (concept) occurs at a given timestamp for a given patient.

Suppose you had a simple time-sorted dataframe as follows:

| subject_id | timestamp           | code                    |
|------------|---------------------|-------------------------|
| 1          | 1989-01-01 00:00:00 | ADMISSION               |
| 1          | 1989-01-01 01:00:00 | LAB&PROCEDURE_START     |
| 1          | 1989-01-01 02:00:00 | DISCHARGE&PROCEDURE_END |
| 2          | 1991-05-06 12:00:00 | ADMISSION               |
| 2          | 1991-05-06 20:00:00 | DEATH                   |
| 3          | 1980-10-17 22:00:00 | ADMISSION&LAB           |
| 3          | 1980-10-18 01:00:00 | LAB                     |
| 3          | 1980-10-18 09:00:00 | DISCHARGE               |
| 3          | 1982-02-02 02:00:00 | ADMISSION               |
| 3          | 1982-02-02 04:00:00 | DEATH                   |

The `CODE` column contains `&` delimited sequences representing the various events that occurred at a given `timestamp` for a given `subject_id`. You may then create a series of predicate columns depending on what suits your needs. For instance, here are some plausible predicate columns that could be created:

| subject_id | timestamp           | code                    | admission | discharge | death | discharge_or_death | lab | procedure_start|
|------------|---------------------|-------------------------|-----------|-----------|-------|--------------------|-----|----------------|
| 1          | 1989-01-01 00:00:00 | ADMISSION               | 1         | 0         | 0     | 0                  | 0   | 0              |
| 1          | 1989-01-01 01:00:00 | LAB&PROCEDURE_START     | 0         | 0         | 0     | 0                  | 1   | 1              |
| 1          | 1989-01-01 02:00:00 | DISCHARGE&PROCEDURE_END | 0         | 1         | 0     | 1                  | 0   | 0              |
| 2          | 1991-05-06 12:00:00 | ADMISSION               | 1         | 0         | 0     | 0                  | 0   | 0              |
| 2          | 1991-05-06 20:00:00 | DEATH                   | 0         | 0         | 1     | 1                  | 0   | 0              |
| 3          | 1980-10-17 22:00:00 | ADMISSION&LAB           | 1         | 0         | 0     | 0                  | 1   | 0              |
| 3          | 1980-10-18 01:00:00 | LAB                     | 0         | 0         | 0     | 0                  | 1   | 0              |
| 3          | 1980-10-18 09:00:00 | DISCHARGE               | 0         | 1         | 0     | 1                  | 0   | 0              |
| 3          | 1982-02-02 02:00:00 | ADMISSION               | 1         | 0         | 0     | 0                  | 0   | 0              |
| 3          | 1982-02-02 04:00:00 | DEATH                   | 0         | 0         | 1     | 1                  | 0   | 0              |

Note that this set of predicates are all `plain` predicates (ie., explicitly expressed as a value in the dataset), with the exception of the `derived` predicate `discharge_or_death`, which can be expressed by applying boolean logic on the `discharge` and `death` predicates (ie., `or(discharge, death)`). You may choose to create these columns for `derived` predicates explicitly (as you would `plain` predicates). Or, ACES can automatically create them from `plain` predicates if the boolean logic is provided in the task configuration file. Please see [Predicates](https://eventstreamaces.readthedocs.io/en/latest/configuration.html#predicates-plainpredicateconfig-and-derivedpredicateconfig) for more information.

## Sample Predicates DataFrame

A sample predicates dataframe is provided in the repository (`../../../sample_data/sample_data.csv`). This dataframe holds completely synthetic data and was designed such that the accompanying sample configuration files in the repository (`../../../sample_configs`) could be directly extracted.

In [ ]:
import polars as pl

pl.read_csv("../../../sample_data/sample_data.csv")

## Generating the Predicates DataFrame

The predicates dataframe will always have the `subject_id` and `timestamp` columns. They should be unique between these two columns, as each row can capture multiple events.

ACES is able to automatically compute the predicates dataframe from your dataset and the fields defined in your task configuration if you are using the `MEDS` or `ESGPT` data standard. Should you choose to not transform your dataset into one of these two currently supported standards, you may also create your own predicates dataframe.

Unfortunately, we are unable to provide code to transform raw datasets into predicates dataframes as data schemas vary widely. However, we provide some sample code below to create the predicates dataframe for the above example as a reference.

In [ ]:
data = pl.DataFrame(
    {
        "patient_id": [1, 1, 1, 2, 2, 3, 3, 3, 3, 3],
        "timestamp": [
            "1/1/1989 00:00",
            "1/1/1989 01:00",
            "1/1/1989 02:00",
            "5/6/1991 12:00",
            "5/6/1991 20:00",
            "10/17/1980 22:00",
            "10/18/1980 01:00",
            "10/18/1980 09:00",
            "2/2/1982 02:00",
            "2/2/1982 04:00",
        ],
        "code": [
            "ADMISSION",
            "LAB&PROCEDURE_START",
            "DISCHARGE&PROCEDURE_END",
            "ADMISSION",
            "DEATH",
            "ADMISSION&LAB",
            "LAB",
            "DISCHARGE",
            "ADMISSION",
            "DEATH",
        ],
    }
).with_columns(pl.col("timestamp").str.strptime(pl.Datetime, format="%m/%d/%Y %H:%M"))

data

We define the function `has_sequence()` to easily create polars expressions needed to search sequences of strings:

In [ ]:
def has_sequence(column: str, sequence: str) -> pl.Expr:
    """Checks if a column contains a sequence.

    Args:
        column (str): The column to search.
        sequence (str): The sequence to search for in the column.

    Returns:
        pl.Expr: A Polars expression representing the check for the sequence.
    """
    has_sequence = pl.col(column).str.contains(sequence)
    return has_sequence

Next, we will first define the `plain` predicates. Suppose we are interested in the following predicates:

- `admission`: realized when code contains the "ADMISSION" sequence, indicating a patient admission
- `discharge`: realized when code contains the "DISCHARGE" sequence, indicating a patient discharge
- `death`: realized when code contains the "DEATH" sequence, indicating a patient death
- `lab`: realized when code contains the "LAB" sequence, indicating a lab for a patient 
- `procedure_start`: realized when code contains the "PROCEDURE_START" sequence, indicating the start of a procedure for a patient

In [ ]:
plain_predicates = {
    "admission": "ADMISSION",
    "discharge": "DISCHARGE",
    "death": "DEATH",
    "lab": "LAB",
    "procedure_start": "PROCEDURE_START",
}

for predicate_name, predicate_value in plain_predicates.items():
    data = data.with_columns(has_sequence("code", predicate_value).alias(predicate_name).cast(pl.Int64))

data

To define columns for `derived` predicates, we can either create them explicitly (similar to how you would columns for `plain` predicates):

In [ ]:
derived_predicates = {
    "discharge_or_death": {"or": ("discharge", "death")},
}

for predicate_name, predicate_value in derived_predicates.items():
    if "or" in predicate_value:
        data = data.with_columns(
            (data[predicate_value["or"][0]] | data[predicate_value["or"][1]])
            .alias(predicate_name)
            .cast(pl.Int64)
        )
    elif "and" in predicate_value:
        data = data.with_columns(
            (data[predicate_value["and"][0]] & data[predicate_value["and"][1]])
            .alias(predicate_name)
            .cast(pl.Int64)
        )

data

Alternatively, you can choose to leverage ACES, which can automatically create `derived` predicate columns from boolean logic in the task configuration file. **Note**: if you are creating `plain` predicate columns directly, you must still define them in the configuration file (could be with an arbitrary value in the `code` field) - ACES will verify their existence after data loading and you will also need them when referencing in your windows.

```yaml
predicates:
  ...
  death:
    code: foo
  discharge:
    code: bar
  discharge_or_death:
    expr: or(discharge, death)
  ...
```
